# Watchlist example notebook

Here we will do a xmatch against a list of positions in the sky, given in the file ```watchlist.csv```

In [1]:
import pandas as pd
import psycopg2

# Get credentials to open a direct connection with the database

In [2]:
import json
# use the location of your credentials file here
credentials_file = "/home/fforster/Dropbox/Work/alercebroker/usecases/alercereaduser_v4.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

In [3]:
conn = psycopg2.connect(dbname=params["dbname"], user=params["user"], host=params["host"], password=params["password"])

# Get watchlist of objects

In [4]:
df = pd.read_csv("data/watchlist.csv")
df.head()

,id_source,ra,dec
0,source_1,160.183014,33.016467
1,source_2,174.215249,44.837895


# Prepare the query string

In [5]:
objects = []
for _,row in df.iterrows():
    objects.append(f"(\'{row.id_source}\', {row.ra}, {row.dec})")
objects_str = ",\n".join(objects)
objects_str

"('source_1', 160.18301441363647, 33.0164673528409),\n('source_2', 174.21524897555543, 44.83789535222221)"

Create query string

In [25]:
search_radius = 0.1 # degrees
query = """
/* First we create the catalog maybe a batch of 1000 objects would be fine? this can be tested
 * The column names oid, ra, dec can be changed but ra and dec has to be in degrees
*/
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
)
/* Then we do the select on object (o) and the new catalog (c), the oid for reference and q3c_dist will calculate
 * the distance, maybe this is expensive, but its useful when we want xmatches and we can filter in the application
 */
SELECT c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec) FROM object o, catalog c
/*
 * Then we do the join REALLY important first use the catalog then the object ra,dec. Don't know but the
 * index is designed that way, if you use other order the query is slooooow. Oh the radius is in degrees too
 */
WHERE q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius});
"""
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

# Send the query and get crossmatching objects

In [26]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(42, 7)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist
0,source_1,160.183014,33.016467,ZTF19aamrxbe,160.240176,33.017146,0.047935
1,source_1,160.183014,33.016467,ZTF19aarfifa,160.282983,32.986167,0.089147
2,source_1,160.183014,33.016467,ZTF20aclohyy,160.168704,33.110273,0.094569
3,source_1,160.183014,33.016467,ZTF20aavpqcf,160.161957,33.051647,0.039360
4,source_1,160.183014,33.016467,ZTF20aautebl,160.190299,33.066016,0.049923
5,source_1,160.183014,33.016467,ZTF19aapkzdk,160.110367,33.061357,0.075657
6,source_1,160.183014,33.016467,ZTF20aazfsjx,160.107502,32.987710,0.069552
7,source_1,160.183014,33.016467,ZTF19aavwgwh,160.118010,33.009265,0.054983
8,source_1,160.183014,33.016467,ZTF20aautebo,160.129415,33.044041,0.052722
9,source_1,160.183014,33.016467,ZTF20aakqkbh,160.167313,32.999504,0.021474
